# 챗봇 만들기 튜토리얼

reference : https://teddylee777.github.io/pytorch/pytorch-seq2seq-chatbot/

Datasets : https://github.com/songys/Chatbot_data/blob/master/ChatbotData.csv

seq2seq_GRU 모델을 Pytorch로 구현 - 한글 챗봇 데이터를 학습시켜 추론해보는 단계까지 진행

In [1]:
import os
import numpy as np
import pandas as pd
import random

In [2]:
df = pd.read_csv("../csv_datasets/ChatbotData.csv")
df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [3]:
question = df['Q']
answer = df['A']

In [4]:
print(question[:5])

0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object


In [5]:
print(answer[:5])

0     하루가 또 가네요.
1      위로해 드립니다.
2    여행은 언제나 좋죠.
3    여행은 언제나 좋죠.
4     눈살이 찌푸려지죠.
Name: A, dtype: object


### 1. 데이터 전처리

##### 1-1. 한글 정규화

In [6]:
import re


# 한글, 영어, 숫자, 공백, ?!.,을 제외한 나머지 문자 제거
korean_pattern = r'[^ ?,.!A-Za-z0-9가-힣+]'

# 패턴 컴파일
normalizer = re.compile(korean_pattern)
normalizer

re.compile(r'[^ ?,.!A-Za-z0-9가-힣+]', re.UNICODE)

In [7]:
print("수정 전 : {}".format(question[11]))
print("수정 후 : {}".format(normalizer.sub("", question[11])))

수정 전 : 가끔 궁금해
수정 후 : 가끔 궁금해


In [8]:
print("수정 전 : {}".format(answer[10]))
print("수정 후 : {}".format(normalizer.sub("", answer[10])))

수정 전 : 자랑하는 자리니까요.
수정 후 : 자랑하는 자리니까요.


In [9]:
def normalize(sentence):
    return normalizer.sub("", sentence)

normalize(question[345])

'금수저로 태어났으면 좋았을텐데'

##### 1-2 한글 형태소 분석기

In [10]:
from konlpy.tag import Okt

# 형태소 분석기
okt = Okt()

In [11]:
okt.morphs(normalize(answer[10]))

['자랑', '하는', '자리', '니까', '요', '.']

In [12]:
# 한글 전처리를 함수화
def clean_text(sentence, tagger):
    
    # print(sentence)
    
    sentence = normalize(sentence)
    # print(sentence)
    
    sentence = tagger.morphs(sentence)
    # print(sentence)
    
    sentence = ' '.join(sentence)
    # print(sentence)
    
    sentence = sentence.lower()
    
    return sentence

In [13]:
clean_text(question[10], okt)

'sns 보면 나 만 빼고 다 행복 해보여'

In [14]:
len(question), len(answer)

(11823, 11823)

In [15]:
questions = [clean_text(sent, okt) for sent in question.values[:1000]]
answers = [clean_text(sent, okt) for sent in answer.values[:1000]]

In [16]:
questions[:5]

['12시 땡 !', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다', '3 박 4일 정도 놀러 가고 싶다', 'ppl 심하네']

In [17]:
answers[:5]

['하루 가 또 가네요 .',
 '위로 해 드립니다 .',
 '여행 은 언제나 좋죠 .',
 '여행 은 언제나 좋죠 .',
 '눈살 이 찌푸려지죠 .']

##### 1-3. 단어 사전 생성

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [19]:
PAD_TOKEN = 0  # 패딩에 사용되는 토큰. (길이가 다른 시퀀스를 동일한 길이로 만들 때 사용)
SOS_TOKEN = 1  # 시작 토큰. 시퀀스의 시작을 나타낸다.
EOS_TOKEN = 2  # 종료 토큰. 시퀀스의 종료를 나타낸다.

# Vocabulary 클래스를 만든다
class WordVocab():
    def __init__(self):
        self.word2index = {  # word2index : 단어를 해당 인덱스에 매핑한다.
            '<PAD>': PAD_TOKEN,  
            '<SOS>': SOS_TOKEN,
            '<EOS>': EOS_TOKEN,
        }
        
        self.word2count = {}  # word2count : 각 단어의 빈도수를 저장한다.
        
        self.index2word = {  # index2word : 인덱스를 해당 단어에 매핑한다.
            PAD_TOKEN: '<PAD>',
            SOS_TOKEN: '<SOS>',
            EOS_TOKEN: '<EOS>'
        }
        
        self.n_words = 3   # 현재 어휘의 크기(단어 수). 초기값은 PAD, SOS, EOS 포함이므로 3.
        
    def add_sentence(self, sentence):  # 문장을 받아서
        for word in sentence.split(' '):  # 문장을 공백 기준으로 단어로 분리한 후,
            self.add_word(word)   # 각 단어를 add_word 메소드에 전달한다
            
    def add_word(self, word):  # 단어를 사전에 추가하는 역할
        if word not in self.word2index:  # 단어가 word2index에 없다면, 단어를 word2index와 index2word에 추가, 
            self.word2index[word] = self.n_words
            self.word2count[word] = 1  # 해당 단어의 빈도를 1로 설정
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count += 1 # 단어가 이미 word2index에 있다면, 해당 단어의 빈도수를 1 증가시킨다.

In [20]:
questions[10]

'sns 보면 나 만 빼고 다 행복 해보여'

In [21]:
print("원문 : {}".format(questions[10]))

lang = WordVocab()
lang.add_sentence(questions[10])
print("==="*10)
print("단어 사전")
print(lang.word2index)
print(lang.word2count)
print(lang.index2word)

원문 : sns 보면 나 만 빼고 다 행복 해보여
단어 사전
{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, 'sns': 3, '보면': 4, '나': 5, '만': 6, '빼고': 7, '다': 8, '행복': 9, '해보여': 10}
{'sns': 1, '보면': 1, '나': 1, '만': 1, '빼고': 1, '다': 1, '행복': 1, '해보여': 1}
{0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: 'sns', 4: '보면', 5: '나', 6: '만', 7: '빼고', 8: '다', 9: '행복', 10: '해보여'}


##### 1-4 padding to sequences

- 하나의 배치 구성을 위해서는 문장의 길이가 맞아야 한다.
- 하지만, 문장 별로 길이가 다르기 때문에 길이를 맞춰 주는 작업을 수행해야 한다.
- 짧은 문장은 남은 공간에 PAD 토큰을 추가하여 길이를 맞춰 주도록 한다.

In [22]:
max_length = 10  # 문장의 최대 길이
sentence_length = 6  # 생성하려는 문장의 길이 정의. 임의로 6짜리 문장 생성한다.

sentence_tokens = np.random.randint(low=3, high=100, size=(sentence_length,)) # 3부터 99 사이의 임의의 정수로 이뤄진 문장 생성. 길이는 6
sentence_tokens = sentence_tokens.tolist() # 리스트로 변환
print("Generated Sentence:{}".format(sentence_tokens))

sentence_tokens = sentence_tokens[:(max_length-1)]  # 문장의 길이가 max_length-1보다 길다면 길이를 줄인다는 뜻
# max_length-1로 하는 이유는 <EOS>를 추가하기 위해서이다.

token_length = len(sentence_tokens)


# 문장의 맨 끝 부분에 <EOS> 토큰 추가
sentence_tokens.append(2)  # <EOS> 토큰의 값은 2

for i in range(token_length, max_length-1):  
    
    # 문장의 길이가 max_length 보다 짧다면, <PAD> 토큰(값 0)으로 문장을 채워서 max length에 맞춘다.
    sentence_tokens.append(0) 
    
print("output: {}".format(sentence_tokens))
print("Total Length : {}".format(len(sentence_tokens)))

Generated Sentence:[99, 47, 27, 62, 67, 8]
output: [99, 47, 27, 62, 67, 8, 2, 0, 0, 0]
Total Length : 10


# 1-5. 전처리 프로세스 클래스화

- torch.utils.data.Dataset을 상속받아 TextDataset 클래스를 구현한다.
- 데이터를 로드하고, 정규화 및 전처리, 토큰화를 진행한다.
- 단어 사전을 생성하고 이에 따라 시퀀스로 변환한다.
- 1-1 ~ 1-4 를 클래스화 한 것

In [24]:
from konlpy.tag import Okt

class TextDataset(Dataset):
    def __init__(self, csv_path, min_length=3, max_length=32):
        super().__init__()
        
        # TOKEN 정의
        self.PAD_TOKEN = 0  # Padding 토큰
        self.SOS_TOKEN = 1  # SOS 토큰
        self.EOS_TOKEN = 2  # EOS 토큰
        
        self.tagger = Okt()  # 형태소 분석기
        self.max_length = max_length  # 한 문장의 최대 길이 지정
        
        
        # CSV 데이터 로드
        df = pd.read_csv("../csv_datasets/ChatbotData.csv")
        
        
        # 한글 정규화
        korean_pattern = r'[^ ?,.!A-Za-z0-9가-힣+]'
        self.normalizer = re.complie(korean_pattern)
        
        
        # src : 질문, tgt : 답변
        src_clean = []
        tgt_clean = []
        
        
        # 단어 사전 생성
        wordvocab = WordVocab()
        
        for _, row in df.iterrows():
            src = row['Q']
            tgt = row['A']
            
            # 한글 전처리
            src = self.clean_text(src)
            tgt = self.clean_text(tgt)
            
            if len(src.split()) > min_length and len(tgt.split()) > min_length:
                # 최소 길이를 넘어가는 문장의 단어만 추가
                wordvocab.add_sentence(src)
                wordvocab.add_sentence(tgt)
                src_clean.append(src)
                tgt_clean.append(tgt)
                
                
        self.srcs = src_clean
        self.tgts = tgt_clean
        self.wordvocab = wordvocab
        
    def normalize(self, sentence):
        # 정규표현식에 따른 한글 정규화
        return self.normalizer.sub("", sentence)
    
    def clean_text(self, sentence):
        # 한글 정규화
        sentence = self.normalize(sentence)
        
        # 형태소 처리
        sentence = self.tagger.morphs(sentence)
        sentence = ' '.join(sentence)
        sentence = sentence.lower()
        return sentence
    
    def texts_to_sequences(self, sentence):
        # 문장 -> 시퀀스로 변환
        return [self.wordvocab.word2index[w] for w in sentence.split()]
    
    def pad_sequence(self, sentence_tokens):
        # 문장의 맨 끝 토큰은 제거
        sentence_tokens = sentence_tokens[:(self.max_length-1)]
        token_length = len(sentence_tokens)
        
        # 문장의 맨 끝 부분에 <EOS> 토큰 추가
        sentence_tokens.append(self.EOS_TOKEN)
        
        for i in range(token_length, (self.max_length-1)):
            # 나머지 빈 곳에 <PAD> 토큰 추가
            sentence_tokens.append(self.PAD_TOKEN)
            
        return sentence_tokens
    
    
    def __getitem__(self, idx):
        inputs = self.srcs[idx]
        inputs_sequences = self.texts_to_sequences(inputs)
        intputs_padded = self.pad_sequence(inputs_sequences)
        
        outputs = self.tgts[idx]
        outputs_sequences = self.texts_to_sequences(outputs)
        outputs_padded = self.pad_sequence(outputs_sequences)
        
        return torch.tensor(inputs_padded), torch.tensor(outputs_padded)
    
    
    def __len__(self):
        return len(self.srcs)
        
        
        
        
        
            

In [3]:
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib.pyplot'